In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
from sklearn.model_selection import train_test_split
import os
import xml.etree.ElementTree as ET
from shutil import copyfile
import argparse
from pathlib import Path
import random

In [ ]:
%pwd

In [ ]:
# train
!wget https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/public_data/IEEE_bigdata_RDD2020/train.tar.gz

!tar -xzvf train.tar.gz
!rm train.tar.gz

In [ ]:
%cd train

In [6]:
os.getcwd()
countries = ["Czech", "India", "Japan"]
labels = ["D00", "D10", "D20", "D40"]

for country in countries:
    annoFiles = os.listdir(os.path.join(os.getcwd(), country+"/annotations/xmls/"))
    jpgFiles = os.listdir(os.path.join(os.getcwd(), country+"/images/"))
    
    newCountry = "new_"+country
    Path(newCountry+"/Annotations").mkdir(parents=True, exist_ok=True)
    Path(newCountry+"/JPEGImages").mkdir(parents=True, exist_ok=True)
    
    for annoFile in annoFiles:
        tree = ET.parse(os.path.join(os.getcwd()+"/"+country+"/annotations/xmls/", annoFile))
        root = tree.getroot()
        for obj in root.findall("object"):
            a = obj.find("name").text
            if a not in labels:
                root.remove(obj)
            
        if len(root.findall("object")) > 0:
            tree.write(newCountry+"/Annotations/"+annoFile)
            copyfile(os.path.join(country+"/images/", annoFile.split(".")[0])+".jpg", newCountry+"/JPEGImages/"+annoFile.split(".")[0]+".jpg")
        else:
            continue

In [7]:
Path("new_train/Annotations").mkdir(parents=True, exist_ok=True)
Path("new_train/JPEGImages").mkdir(parents=True, exist_ok=True)

!cp new_Czech/Annotations/*.xml new_train/Annotations/
!cp new_India/Annotations/*.xml new_train/Annotations/
!cp new_Japan/Annotations/*.xml new_train/Annotations/

!cp new_Czech/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_India/JPEGImages/*.jpg new_train/JPEGImages/
!cp new_Japan/JPEGImages/*.jpg new_train/JPEGImages/

In [8]:
%cd ..

/content


In [9]:
annoFiles = os.listdir(os.path.join(os.getcwd(), "train/new_train/Annotations/"))
yoloFile = open("./xml2yolo_damage.txt", "w")
for i in range(len(annoFiles)):
    yoloFile.writelines(os.getcwd() + "/train/new_train/Annotations/" + annoFiles[i] + "\n")

yoloFile.close()

In [ ]:
!cp gdrive/MyDrive/colabFiles/damage.names /content
!cp gdrive/MyDrive/colabFiles/xml2yolo.py /content
!python xml2yolo.py --input-file xml2yolo_damage.txt

In [11]:
Path("road/data").mkdir(parents=True, exist_ok=True)

!cp train/new_train/JPEGImages/*.jpg road/data
!cp train/new_train/labels/*.txt road/data

In [12]:
!ls

damage.names  road	   train		xml2yolo.py
gdrive	      sample_data  xml2yolo_damage.txt


In [14]:
drivefl0 = os.listdir(os.path.join(os.getcwd(), "train/new_train/Annotations"))
print(len(drivefl0))

12195


In [15]:
drivefl0 = os.listdir(os.path.join(os.getcwd(), "road/data"))
print(len(drivefl0))

24390


In [24]:
%cd train

/content/train


In [25]:
new_Czech_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Czech/JPEGImages/"))
print(len(new_Czech_annoFiles))

new_India_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_India/JPEGImages/"))
print(len(new_India_annoFiles))

new_Japan_annoFiles = os.listdir(os.path.join(os.getcwd(), "new_Japan/JPEGImages/"))
print(len(new_Japan_annoFiles))

Czech_train, Czech_valid = train_test_split(new_Czech_annoFiles, test_size=0.2, random_state=42)
print(len(Czech_train), len(Czech_valid))

India_train, India_valid = train_test_split(new_India_annoFiles, test_size=0.2, random_state=42)
print(len(India_train), len(India_valid))

Japan_train, Japan_valid = train_test_split(new_Japan_annoFiles, test_size=0.2, random_state=42)
print(len(Japan_train), len(Japan_valid))

1072
3223
7900
857 215
2578 645
6320 1580


In [19]:
strpath ='/data/'

# single source training dataset

file1 = open("../road/Cz8.txt", "w")
for i in range(len(Czech_train)):
    file1.writelines(strpath + Czech_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Cz2val.txt", "w")
for i in range(len(Czech_valid)):
    file2.writelines(strpath + Czech_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

file1 = open("../road/In8.txt", "w")
for i in range(len(India_train)):
    file1.writelines(strpath + India_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/In2val.txt", "w")
for i in range(len(India_valid)):
    file2.writelines(strpath + India_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

file1 = open("../road/Ja8.txt", "w")
for i in range(len(Japan_train)):
    file1.writelines(strpath + Japan_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Ja2val.txt", "w")
for i in range(len(Japan_valid)):
    file2.writelines(strpath + Japan_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

In [20]:
# mix source training dataset

CzInJa_train = new_Czech_annoFiles + new_India_annoFiles + Japan_train
random.shuffle(CzInJa_train)

CzInJa_valid = Japan_valid
random.shuffle(CzInJa_valid)

file1 = open("../road/CzInJa.txt", "w")
for i in range(len(CzInJa_train)):
    file1.writelines(strpath + CzInJa_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/CzInJaval.txt", "w")
for i in range(len(CzInJa_valid)):
    file2.writelines(strpath + CzInJa_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

japanHalfLength = len(new_Japan_annoFiles)/2

JaCz_train = new_Japan_annoFiles[:int(japanHalfLength)] + Czech_train
random.shuffle(JaCz_train)

file1 = open("../road/Ja5Cz8.txt", "w")
for i in range(len(JaCz_train)):
    file1.writelines(strpath + JaCz_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Ja5Cz8val.txt", "w")
for i in range(len(Czech_valid)):
    file2.writelines(strpath + Czech_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

JaIn_train = new_Japan_annoFiles[:int(japanHalfLength)] + India_train
random.shuffle(JaIn_train)

file1 = open("../road/Ja5In8.txt", "w")
for i in range(len(JaIn_train)):
    file1.writelines(strpath + JaIn_train[i].split(".")[0] + '.jpg' + "\n")

file1.close()

file2 = open("../road/Ja5In8val.txt", "w")
for i in range(len(India_valid)):
    file2.writelines(strpath+ India_valid[i].split(".")[0] + '.jpg' + "\n")

file2.close()

In [ ]:
%cd ..

In [ ]:
!zip -r /content/road.zip /content/road